In [1]:
import pandas as pd
pd.set_option('display.max_columns', 100)
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib notebook 

In [2]:
# On charge les dataframe
df_train = pd.read_csv('sources/train.csv')
df_test = pd.read_csv('sources/test.csv')

In [ ]:
!pip install pandas_profiling

     |████████████████████████████████| 133kB 759kB/s eta 0:00:01
     |████████████████████████████████| 614kB 594kB/s eta 0:00:01
     |████████████████████████████████| 6.3MB 810kB/s eta 0:00:01     |██████████▊                     | 2.1MB 521kB/s eta 0:00:09
     |████████████████████████████████| 235kB 695kB/s eta 0:00:01
     |████████████████████████████████| 307kB 828kB/s eta 0:00:01
     |████████████████████████████████| 92kB 1.0MB/s eta 0:00:01


     |████████████████████████████████| 51kB 1.0MB/s eta 0:00:01
     |████████████████████████████████| 215kB 794kB/s eta 0:00:01
     |███████████████████████▏        | 40kB 727kB/s eta 0:00:01

In [ ]:
import pandas_profiling
df_train.profile_report(style={'full_width':True})

In [ ]:
numerical_feats = df_train.dtypes[df_train.dtypes != "object"].index
print("Number of Numerical features: ", len(numerical_feats))
print("-"*79)
categorical_feats = df_train.dtypes[df_train.dtypes == "object"].index
print("Number of Categorical features: ", len(categorical_feats))
print("-"*79)
print(df_train[numerical_feats].columns)
print("-"*79)
print(df_train[categorical_feats].columns)

In [ ]:
# Plot bar plot for each categorical feature

for feature in df_train.dtypes[df_train.dtypes == 'object'].index:
    sns.countplot(y=feature, data=df_train)
    plt.show()

In [ ]:
sns.distplot(df_train['SalePrice']);

In [ ]:
df_train['SalePrice_Log'] = np.log1p(df_train['SalePrice'])
sns.distplot(df_train['SalePrice_Log']);

In [ ]:
total = df_train.isnull().sum().sort_values(ascending=False)
percent = (df_train.isnull().sum()/df_train.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

In [ ]:
# columns where NaN values have meaning e.g. no pool etc.
cols_fillna = ['PoolQC','MiscFeature','Alley','Fence','MasVnrType','FireplaceQu',
               'GarageQual','GarageCond','GarageFinish','GarageType', 'Electrical',
               'KitchenQual', 'SaleType', 'Functional', 'Exterior2nd', 'Exterior1st',
               'BsmtExposure','BsmtCond','BsmtQual','BsmtFinType1','BsmtFinType2',
               'MSZoning', 'Utilities']

# replace 'NaN' with 'None' in these columns
for col in cols_fillna:
    df_train[col].fillna('None',inplace=True)
    df_test[col].fillna('None',inplace=True)

In [ ]:
total = df_train.isnull().sum().sort_values(ascending=False)
percent = (df_train.isnull().sum()/df_train.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(5)

In [ ]:
# fillna with mean or mode for the remaining values
df_train.fillna(df_train.mean(), inplace=True)
df_test.fillna(df_test.mean(), inplace=True)
# df_train.fillna(df_train.mode(), inplace=True) mode -> la valeur la plus représentée dans le dataset
# df_test.fillna(df_test.mode(), inplace=True)

In [ ]:
df_train.isnull().sum().sum()

In [ ]:
df_test.isnull().sum().sum()

Split dataset

In [ ]:
# Split
from sklearn.model_selection import train_test_split

X_train, X_val = train_test_split(df_train, test_size=0.2, random_state=42)

In [ ]:
X_train_num = X_train[numerical_feats]

Standardize and Normalize Data

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_train_num)
X_scaled

In [ ]:
type(X_scaled)

In [ ]:
pca = PCA(n_components=2)
pca.fit_transform(df_train[numerical_feats])
print(f"Axes : {pca.explained_variance_ratio_}")
print(f"Sum : {sum(pca.explained_variance_ratio_)}")
print(pca.singular_values_) 

In [ ]:
pca.components_

In [ ]:
# Split
from sklearn.model_selection import train_test_split
X_train, X_val = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
X_train.head(5)

In [ ]:
# y_train = X_train['SalePrice'].values
# X_train = X_train.drop('SalePrice', axis=1).values

In [ ]:
X_train_num = X_train.select_dtypes(include=['number']).copy()

In [ ]:
X_train_num.head(5)

In [ ]:
from sklearn import preprocessing
mm_scaler = preprocessing.MinMaxScaler()
X_train = mm_scaler.fit_transform(X_train_num)

In [ ]:
X_train_num

In [ ]:
X_train = pd.DataFrame(data=X_train[1:,1:], index=X_train[1:,0], columns=X_train[0,1:])
X_train

In [ ]:
Y = X_train['SalePrice']
X = X_train.drop('SalePrice', axis=1)

In [ ]:
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(X, Y)